In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
from spacy.matcher import PhraseMatcher
from collections import defaultdict
from IPython.core.display import display, HTML

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

kaggle/input\covid19-literature\df_covid.csv


In [9]:
data = pd.read_csv("kaggle/input/covid19-literature/df_covid.csv")
data.head()

,paper_id,abstract,body_text,authors,title,journal,abstract_summary,abstract_word_count,body_word_count
0,25621281691205eb015383cbac839182b838514f,the human interferon ifninduced mxa protein is...,influenza a viruses iav are severe human patho...,"Dornfeld, Dominik. Dudek, Alexandra H....",SMARCA2-regulated host cell factors are<br>re...,Sci Rep,The human interferon (IFN)-induced MxA<br>pro...,924,5136
1,7db22f7f81977109d493a0edf8ed75562648e839,scorpine a small cationic peptide from the ven...,the oldest known scorpions lived around 430 mi...,"Zhang, Chao. He, Xinlong...",Recombinant Scorpine Produced Using SUMO<br>F...,PLoS One,"Scorpine, a small cationic peptide from the<b...",253,3543
2,a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a,background the complex interplay between viral...,the emergence of severe acute respiratory synd...,"McDermott, Jason E.. Mitchell, Hugh D....",The effect of inhibition of PP1 and TNFα<br>s...,BMC Syst Biol,Background: The complex interplay between<br>...,255,5137
3,6c3e1a43f0e199876d4bd9ff787e1911fd5cfaa6,NaN,sjgrens syndrome ss is a connective tissue dis...,"Talotta, Rossella. Sarzi-Puttini, Piercarlo...",Microbial Agents as Putative Inducers of B Ce...,J Immunol Res,Not provided.,0,5977
4,2ce201c2ba233a562ee605a9aa12d2719cfa2beb,background human adenovirus type 55 is a reeme...,human adenovirus hadv is a common pathogen amo...,"Yi, Lina. Zou, LiRong...",A cluster of adenovirus type B55 infection in...,Influenza Other Respir Viruses,Background: Human adenovirus type 55 is a<br>...,250,1289


# Intro to NLP using SpaCy as taught in the Kaggle mini course

This is mostly a copy and paste job so certain variable names may not make sense.

In [10]:
# Full term list.
menu = ["infection","incubation","environmental stability",]

# To limit searches to a partial term list.
menu = menu[:3]

# Load the SpaCy model
nlp = spacy.blank('en')
nlp.max_length = 2000000

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

matcher.add("MENU",            # Just a name for the set of rules we're matching to
            None,              # Special actions to take on matched words
            *menu_tokens_list)

In [7]:
# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)

for idx, review in data.iterrows():
    doc = nlp(review["body_text"])
    # Using the matcher from the previous exercise
    matches = matcher(doc)
    
    # Create a set of the items found in the review text
    found_items = [{"item":doc[item[1]:item[2]],
                    "match_excerpt":doc[item[1]-50:item[2]+50]}
                   for item in matches]
        
    # Update item_ratings with rating for each item in found_items
    # Transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[str(item["item"]).lower()].append({"title":review["title"],
                                                        "authors":review["authors"],
                                                        "journal":review["journal"],
                                                        "index":idx,
                                                        "id":review["paper_id"],
                                                        "matches":len(matches),
                                                        "match_excerpt":item["match_excerpt"]})

# Display search results

In [5]:
for item in item_ratings.items():
    display(HTML(f'<h1>{item[0].upper()}</h1>'))
    for i in item[1]:
        marked_excerpt = str(i["match_excerpt"]).replace(item[0], f"<mark>{item[0]}</mark>")
        display(HTML(f"""<p>
                <strong>Title:</strong> {i["title"]}</br>
                <strong>Authors:</strong> {i["authors"]}</br>
                <strong>Journal:</strong> {i["journal"]}</br>
                <strong>Item:</strong> {item[0].capitalize()}</br>
                <strong>Index:</strong> {i["index"]}</br>
                <strong>ID:</strong> {i["id"]}</br>
                <strong>Number of matches:</strong> {i["matches"]}
            </p>
            <blockquote>...{marked_excerpt}...</blockquote>"""))

NameError: name 'item_ratings' is not defined